# Data

## Image

Ảnh lấy từ các trạm giám sát video
* Trạm **CFA**: 5 camera cao 30m
* Trạm **BCN**: 6 camera cao 142m
* 3 loại hình ảnh mỗi giờ vào ban ngày:
    * ảnh chụp nhanh (snapshot)
    * ảnh phơi sáng thời gian (timex)
    * ảnh phương sai (variance)
* Môi trường **không có thủy triều $\rightarrow$ $z = z_{msl}$ (mean sea level)**
* hơn 15 ground control points để hiệu chuẩn ảnh từ camera
* **2pixels/meter**

## Label

* **143 planviews** đường bở biển được đánh dấu bởi **3 chuyên gia** để nghiên cứu sự biến đổi **trong năm** và **trong ngày**:
    * Trong năm: 1 ảnh vào trưa ngày 15 mỗi tháng từ 2015 $\rightarrow$ 2017
    * Trong ngày: 18 ngày khác nhau (chủ yếu vào mua đông để kiểm tra trong đk sóng lớn, độ cao mặt trời thấp)

* RMSE của 3 experts tương đối thấp, khoảng **2-3pixel ~ 1-1.5m**

# Automatic Shoreline Detection


## Raw Shorelines

**4 different channels** of planview image được sử dụng:
* 🎨 **Hue (Sắc độ)**: Biểu thị màu sắc, có thể giúp phân biệt màu nước biển (thường xanh lam hoặc lục) với màu đất liền (thường vàng, nâu hoặc xanh lục đậm).
* 🌈 **Saturation (Độ bão hòa)**: Đo độ tinh khiết của màu sắc. Nước biển thường có độ bão hòa thấp hơn so với đất liền
* 💡**Value (Độ sáng)**: Xác định mức độ sáng của pixel, giúp phân biệt giữa các vùng sáng như bãi cát và vùng tối hơn như mặt nước sâu (sử dụng SLIM algorithm, rất hữu ích khi có ánh năng chói)
* 🔴🟢**Red/Green**:
    * Red:  Đất liền, đặc biệt là cát và thực vật, thường có giá trị cao hơn trong kênh đỏ so với nước biển.
    * Green: Nước biển có thể có giá trị cao hơn trong kênh xanh lá do tính phản xạ của nước.

👀 Cách **detect shoreline với từng kênh**:
* **Hue**: gradient cực đại từ đất ra biển
* **Saturation**: gradient cực đại từ biển vào đất
* **Value(HSV)**: gradient tuyệt đối cực đại
* **Red/Green**: gradient tuyệt đối cực đại

⚠️ Các giá trị này được làm mượt bằng **bộ lọc Butterworth** với độ dài đặc trưng 15m để giảm nhiễu

## Weighted Combination of the Raw Shorelines

### Combination

**1️⃣ Xây dựng ma trận raw shorelines**

Ký hiệu:  $D_{ij}^k$ với:
* $k$ là chỉ số phương pháp (Hue, Saturation, Value, R/G)
* $i$ là chỉ số transect (đường cắt vuông góc với bờ biển)
* $j$ là thời điểm quan sát

**2️⃣ Làm mượt dữ liệu bằng Butterwoth**

* tạo ra một bản làm mượt: $\bar{D}_{ij}^k$
* tính sai số để đo độ tin cậy:
$$ e_{ij}^k = | D_{ij}^k - \bar{D}_{ij}^k | $$

> Độ lệch tuyệt đối

3️⃣ **Tính trọng số cho từng phương pháp**

* Gán trọng số $w_{ij}^k$ cho từng phương pháp:
$$ w_{ij}^k = \frac{1}{(e_{ij}^k + \epsilon)^2} $$

trong đó $\epsilon = 10^{-6}px$ để tránh chia cho 0

> 💡 Sai số càng nhiều thì trọng số càng bé 

4️⃣ **Kết hợp các đường bờ biển thô**

* Lấy trung bình có trọng số của các phương pháp:
$$ D_{ij}^c = \frac{\sum_k w_{ij}^kD_{ij}^k}{\sum_kw_{ij}^k} $$

### Error (check quality)

1️⃣**Sai số phương pháp thô với đường bờ kết hợp**:

$$ E_{ij}^{c0} = \sqrt{ \frac{\sum_k w_{ij}^k(D_{ij}^k - D_{ij}^c)^2}{\sum_kw_{ij}^k} } \tag{1}$$

2️⃣**Sai số về độ mượt của đường bờ**

$$ E_{ij}^{c1} = |D_{ij}^k - D_{ij}^c|\tag{2}$$

3️⃣**Tổng sai số**:  
$$ (1) + (2): E_{ij}^c = E_{ij}^{c0} + E_{ij}^{c1} $$

> set ngưỡng sai số tối đa cho phép: $E^c < 10px$ nếu không thỏa mãn thì bỏ

### Filtering

*Lọc không gian và thời gian*

1️⃣ **Trọng số** cho từng điểm:  
$$ w_{ij}^c = \frac{1}/{(E_{ij}^c + \epsilon)^2} $$

2️⃣ **Lọc theo thời gian**:  

*Sử dụng trung bình động $l$ thời điểm gần nhất*:
$$ D_{ij}^{f0} = \frac{\sum_l w_{il}^c D_{il}^c}{\sum_l w_{il}^c} $$

* $l$ là các thời điểm gần $j$

> loại bỏ nhiễu do ánh sáng thay đổi theo thời gian

3️⃣ **Lọc theo không gian**:  

*Sử dụng trung bình động $l$ transect gần nhất*:
$$ D_{ij}^{f} = \frac{\sum_l w_{lj}^c D_{lj}^{f0}}{\sum_l w_{lj}^c} $$

⚠️ Lưu ý là lấy D sau lọc thời gian để lọc không gian

4️⃣**Tổng hợp sai số**:  

*Trung bình của ss không gian với thời gian:*  
$$ E^f_{ij} = \frac{1}{2} \left[ \frac{\sum_l w^c_{il} E^c_{il}}{\sum_l w^c_{il}} + \frac{\sum_l w^c_{lj} E^c_{lj}}{\sum_l w^c_{lj}} \right]
 $$

# Xem lại cách lấy điểm và bỏ điểm

# Trang 11

# Results